In [ ]:
import numpy as np
from sklearn.cross_decomposition import CCA
import matplotlib.pyplot as plt

In [ ]:
num_bins = 10
bin_length = 0.1
num1 = 50
num2 = 30

In [ ]:
class Neuron:
    def __init__(self, gen):
        self.firing_rate = max(0.5, gen.lognormal(0.2, 2.5))
    
    def sample_bins(self, num_bins, bin_length, gen):
        return gen.poisson(self.firing_rate * bin_length, size=num_bins)


In [ ]:
def generate_neurons(num_neurons, gen):
    return [Neuron(gen) for _ in range(num_neurons)]

def generate_binned_trial_rates(neurons, num_bins, bin_length, gen):
    return np.array([neuron.sample_bins(num_bins, bin_length, gen) for neuron in neurons])

In [ ]:
gen = np.random.default_rng(0)

neurons_r1 = generate_neurons(num1, gen)
neurons_r2 = generate_neurons(num2, gen)

binned_trial_rates_r1 = generate_binned_trial_rates(neurons_r1, num_bins, bin_length, gen)
binned_trial_rates_r2 = generate_binned_trial_rates(neurons_r2, num_bins, bin_length, gen)

In [ ]:
binned_trial_rates_r1

In [ ]:
def fit_cca(binned_trial_rates_r1, binned_trial_rates_r2, cca=None, fit=True):
    if cca is None:
        cca = CCA(n_components=1, scale=False)
    if fit:
        cca.fit(binned_trial_rates_r1, binned_trial_rates_r2)
    X, Y = cca.transform(binned_trial_rates_r1, binned_trial_rates_r2)
    X = X.squeeze()
    Y = Y.squeeze()
    return cca, X, Y


In [ ]:
def find_correlation(X, Y):
    if len(X.shape) == 1:
        return np.corrcoef(X, Y)[0, 1]
    else:
        return np.array([np.corrcoef(x, y)[0, 1] for x, y in zip(X.T, Y.T)])

In [ ]:
binned_trial_r1 = generate_binned_trial_rates(neurons_r1, num_bins, bin_length, gen)
binned_trial_r2 = generate_binned_trial_rates(neurons_r2, num_bins, bin_length, gen)

In [ ]:
# Option number 1
n_comp = 1
cca, X, Y = fit_cca(binned_trial_rates_r1.T, binned_trial_rates_r2.T, cca=CCA(n_components=n_comp))
print(f"Correlation: {find_correlation(X, Y)}")
cca, X, Y = fit_cca(binned_trial_r1.T, binned_trial_r2.T, cca=cca, fit=True)
print(f"Correlation: {find_correlation(X, Y)}")
    

In [ ]:
only_frates_r1 = np.array(
    [generate_binned_trial_rates(neurons_r1, 1, 1, gen).squeeze() for _ in range(100)]
)
only_frates_r2 = np.array(
    [generate_binned_trial_rates(neurons_r2, 1, 1, gen).squeeze() for _ in range(100)]
)

In [ ]:
only_frates_r1

In [ ]:
only_frates_r2

In [ ]:
# Option number 3
cca, X, Y = fit_cca(only_frates_r1[:50], only_frates_r2[:50], cca=CCA(n_components=1))
for val1, val2 in zip(X, Y):
    print(val1, val2)
print(np.corrcoef(X, Y))
plt.scatter(X, Y)
cca, X, Y = fit_cca(only_frates_r1[50:], only_frates_r2[50:], cca=cca, fit=False)
print(np.corrcoef(X, Y))
plt.scatter(X, Y)

In [ ]:
plt.scatter(X, Y)

In [ ]:
cca.get_params()

In [ ]:
X

In [ ]:
Y

In [ ]:
np.corrcoef(X, Y)[0, 1]

In [ ]:
only_frates_a = np.array(
    [generate_binned_trial_rates(neurons_r1, 1, 1, gen).squeeze() for _ in range(100)]
)
only_frates_b = np.array(
    [generate_binned_trial_rates(neurons_r2, 1, 1, gen).squeeze() for _ in range(100)]
)

In [ ]:
cca, X, Y = fit_cca(only_frates_a, only_frates_b, cca=cca, fit=False)
print(np.corrcoef(X, Y)[0, 1])

In [ ]:
# Option number 2 - seems to follow paper
per_trial_bins1 = np.concatenate(
    [generate_binned_trial_rates(neurons_r1, num_bins, bin_length, gen).T for _ in range(100)], axis=0
)
per_trial_bins2 = np.concatenate(
    [generate_binned_trial_rates(neurons_r2, num_bins, bin_length, gen).T for _ in range(100)], axis=0
)

print(per_trial_bins1.shape)
print(per_trial_bins2.shape)

In [ ]:
cca, Xtrain, Ytrain = fit_cca(per_trial_bins1[:500], per_trial_bins2[:500], cca=CCA(n_components=1), fit=True)
cca, Xtest, Ytest = fit_cca(per_trial_bins1[500:], per_trial_bins2[500:], cca=cca, fit=False)

In [ ]:
Xtrain.shape

In [ ]:
for val in range(10, 510, 10):
    start = val - 10
    print(f"Test {val} : {np.corrcoef(Xtrain[start:val], Ytrain[start:val])[0, 1]}")
    print(f"Train {val} : {np.corrcoef(Xtest[start:val], Ytest[start:val])[0, 1]}")